In [1]:
####
#Author: alan leidner
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time

from selenium import webdriver



In [2]:
####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome()
#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)


<selenium.webdriver.remote.webelement.WebElement (session="77f1085add81f6a455458d282bee577b", element="de64d178-cb18-4534-98cc-fc32caba244d")>


In [3]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
5,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
6,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
7,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
8,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
9,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY


In [4]:
##DELETE BLANK ROW##
df.drop([0], inplace=True)
df.head()
display(df)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
5,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
6,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
7,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
8,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
9,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY
10,1000 Islands Hose Haulers,45-38-38,454570241,NFP,CARTHAGE,NY


In [5]:
#Select "Next" in test browser, and load table

inputElement2 = browser.find_element_by_partial_link_text("Next").click()
table2 = browser.find_element_by_css_selector('table.Bordered')

print(table2)


<selenium.webdriver.remote.webelement.WebElement (session="77f1085add81f6a455458d282bee577b", element="a34f718e-73c5-449f-92a9-635e3eca9b42")>


In [6]:
#Load 2nd table into a df
df2 =[]
## loop through dataframe to export table
for row in table2.find_elements_by_css_selector('tr'):
      cols = df2.append([cell.text for cell in row.find_elements_by_css_selector('td')])

df2 = pd.DataFrame(df2, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
df2.drop([0], inplace=True)
display(df2)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,100 CLUB OF BUFFALO INC,03-01-89,160957291,NFP,BUFFALO,NY
2,100 E. 182ND Street Housing Development Fund C...,43-46-29,133881096,NFP,BRONX,NY
3,100 GROTON PARKWAY INC,15-51-96,222661001,NFP,ROCHESTER,NY
4,"100 Lives Foundation, Inc.",45-60-79,364808338,NFP,WASHINGTON,DC
5,"100 Mile Man Foundation, Inc.",20-23-17,200565515,NFP,NEW YORK,NY
6,100 WHO CARE INC,06-65-86,134068307,NFP,LONG ISLAND CITY,NY
7,"100 Women in Finance Association, Inc.",20-90-57,571174550,NFP,NEW YORK,NY
8,1026 Foundation Corporation (The),46-42-92,823378707,NFP,SPRINGFIELD GARDENS,NY
9,1035 BROADWAY HOUSING DEVELOPMENT FUND COMPANY...,05-75-26,133870736,NFP,MT. KISCO,NY
10,"103rd Precinct Community Council, Inc.",41-40-17,205484666,NFP,JAMAICA,NY


In [7]:
df = df.append(df2, ignore_index=True)
df.head(30)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
1,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
2,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
3,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
4,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
5,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
6,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
7,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
8,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY
9,1000 Islands Hose Haulers,45-38-38,454570241,NFP,CARTHAGE,NY


In [8]:
#loop through next pages to collect data
#(browser.find_element_by_partial_link_text("Next") = True):
while (browser.find_element_by_partial_link_text("Next").get_attribute('href') is not None):
    inputElementi = browser.find_element_by_partial_link_text("Next").click()
    tablei = browser.find_element_by_css_selector('table.Bordered')
    dfi =[]
## loop through table to load into dataframe
    for row in tablei.find_elements_by_css_selector('tr'):
        cols = dfi.append([cell.text for cell in row.find_elements_by_css_selector('td')])
    dfi = pd.DataFrame(dfi, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
    dfi.drop([0], inplace=True)
    df = df.append(dfi, ignore_index=True)
##I should find a graceful way to exit this later, but it works!

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=89.0.4389.114)


In [10]:
display(df)
##close test browser
browser.quit()

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
1,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
2,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
3,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
4,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
...,...,...,...,...,...,...
95,University of Virginia Health Foundtion,40-44-88,412097394,NFP,CHARLOTTESVILLE,VA
96,Violin Player,41-40-19,270773158,NFP,EAST AMHERST,NY
97,"William A. Epps Community Center, Inc.",40-91-11,861074714,NFP,STATEN ISLAND,NY
98,WORLD SOCIETY OF CZESTOCHOWA JEWS AND THEIR DE...,40-46-49,205101779,NFP,NEW YORK,NY


In [11]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3:/m10-database-update-bucket-v2/'#specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3:/m10-database-update-bucket-v2/charities_bureau_scrape_20210406202759.csv


In [ ]:
## Links to csv's in s3##
## s3://m10-database-update-bucket-v2/charities_bureau_scrape_20210402164723.csv ## First page 
## s3://m10-database-update-bucket-v2/charities_bureau_scrape_20210402165040.csv ## Cleared null row        
## s3://m10-database-update-bucket-v2/charities_bureau_scrape_20210406202759.csv ## Dataframe from all pages  